In [1]:
import pandas as pd
import numpy as np

In [2]:
# loading in data
drugs_2012 = pd.read_stata('2012.dta', convert_categoricals=False) #2012 example

In [3]:
drugs_2012.head()

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,IIEMPSTY,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,ANALWT_C,VESTR,VEREP
0,1,50886467,2,4,4,991,9991,91,91,91,...,1,1,99,9,9,2,2,1275.60,30054,2
1,2,13766883,2,99,99,991,9991,91,91,91,...,1,1,1,1,1,2,2,5191.07,30031,1
2,3,17772877,2,99,99,991,9991,91,91,91,...,1,1,1,1,1,3,3,419.74,30056,2
3,4,45622817,1,99,99,13,9999,99,2,93,...,1,1,2,1,1,2,2,1449.30,30054,1
4,5,17239390,1,99,99,11,9999,99,4,93,...,1,1,1,1,1,1,1,15344.29,30012,2


In [4]:
drugs_2012.shape

(55268, 3120)

# Converting Religion Columns

In [5]:
drugs_2012 = drugs_2012.set_index('CASEID')
drugs_2012.head()

,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,...,IIEMPSTY,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,ANALWT_C,VESTR,VEREP
CASEID,,,,,,,,,,,,,,,,,,,,,
1,50886467,2,4,4,991,9991,91,91,91,91,...,1,1,99,9,9,2,2,1275.60,30054,2
2,13766883,2,99,99,991,9991,91,91,91,91,...,1,1,1,1,1,2,2,5191.07,30031,1
3,17772877,2,99,99,991,9991,91,91,91,91,...,1,1,1,1,1,3,3,419.74,30056,2
4,45622817,1,99,99,13,9999,99,2,93,93,...,1,1,2,1,1,2,2,1449.30,30054,1
5,17239390,1,99,99,11,9999,99,4,93,93,...,1,1,1,1,1,1,1,15344.29,30012,2


Columns for me to look through:

1. `SNRLGSVC`: PAST 12 MOS, HOW MANY RELIG. SERVICES
2. `SNRLGIMP`: MY RELIGIOUS BELIEFS ARE VERY IMPORTANT
3. `SNRLDCSN`: MY RELIGIOUS BELIEFS INFLUENCE MY DECISIONS
4. `SNRLFRND`: IT IS IMPORTANT THAT MY FRIENDS SHARE REL BELIEFS

In [6]:
# nested dictionary for the values we will convert each column to 
religion_dict = {'SNRLGSVC':
             {1:'0',
              2:'1-2',
              3:'3-5',
              4:'6-24',
              5:'25-52',
              6:'52+'},

             'SNRLGIMP':
             {1:'Strongly Disagree',
              2:'Disagree',
              3:'Agree',
              4:'Strongly Agree'},

             'SNRLDCSN':
              {1:'Strongly Disagree',
              2:'Disagree',
              3:'Agree',
              4:'Strongly Agree'},

             'SNRLFRND':
             {1:'Strongly Disagree',
              2:'Disagree',
              3:'Agree',
              4:'Strongly Agree'}
            }

In [7]:
def endcode_col(df, col):
    '''
    Encodes the column values of the dataframe to more readable description.
    
    param df: DataFrame
    param col: Column name
    return: None
    '''
    temp_dict = religion_dict[col]
    df[col] = df[col].apply(lambda value: temp_dict[value] if value != 99 else '99')

In [8]:
def convert_religion_cols(df):
    '''
    Converts the religion columns into a more readable description and return a new DataFrame.
    
    param df: DataFrame
    return: DataFrame with only religious columns
    '''
    religion_cols = ['SNRLGSVC','SNRLGIMP','SNRLDCSN','SNRLFRND'] # columns related to religion
    religion_df = df[religion_cols] # creating new df with only religion_cols 

    # converting bad/missing data to one value
    religion_df = religion_df.applymap(lambda x : 99 if x <= 99 and x >= 94 else x)
    
    # converting relgion column values 
    for col in religion_df.columns:
        endcode_col(religion_df, col)
        
    return religion_df

In [9]:
convert_religion_cols(drugs_2012)

,SNRLGSVC,SNRLGIMP,SNRLDCSN,SNRLFRND
CASEID,,,,
1,99,99,99,99
2,1-2,Agree,Agree,Disagree
3,52+,Strongly Agree,Strongly Agree,Disagree
4,6-24,Agree,Agree,Strongly Disagree
5,1-2,Strongly Disagree,Strongly Disagree,Strongly Disagree
...,...,...,...,...
55264,0,Agree,Agree,Agree
55265,99,99,99,99
55266,0,99,99,99
